In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pandasql import sqldf
import numpy as np
data=pd.read_csv('../input/house-price-prediction/train.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
print(data.nunique())

In [ ]:
data.info()

In [ ]:
# Convert year column to number and calculate the age for the column YearBuilt, YearRemodAdd, GarageYrBlt, YrSold
data['AgeYearBuilt'] = data.YearBuilt.max() - data['YearBuilt']
data['AgeYearRemodAdd'] = data.YearRemodAdd.max() - data['YearRemodAdd']
data['AgeGarageYrBlt'] = data.GarageYrBlt.max() - data['GarageYrBlt']
data['AgeYrSold'] = data.YrSold.max() - data['YrSold']

# drop the original column as we will use above created column
data.drop(['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1,inplace=True)

In [ ]:
data.describe()

In [ ]:
#PoolQC,MiscFeature,Alley,Fence,FirePlaceQu - these columns should be dropped since most of the values are null

Cdata = data.drop(columns=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','Id'],axis=1)

In [ ]:
#pysqldf = lambda q: sqldf(q, globals())

q = """SELECT distinct MSZoning FROM Cdata LIMIT 10;"""

print(sqldf(q))

In [ ]:
#MISSING DATA TREATMENT

cat = []
con = []

for i in Cdata.columns:
    if Cdata[i].dtype=='object':
        cat.append(i)
    else:
        con.append(i)

In [ ]:
#Univariate and Bivariate Analysis
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plotindex = 0
fig,ax = plt.subplots(1,3,figsize=(15,5))
for index,i in enumerate(con): 
    if  plotindex <3:
        ax[plotindex].scatter(Cdata[i],Cdata['SalePrice'])
        ax[plotindex].set(xlabel=i, ylabel='SalePrice')
        plotindex = plotindex+1
    else:
        plotindex=0
        fig,ax = plt.subplots(1,3,figsize=(15,5))
        ax[plotindex].scatter(Cdata[i],Cdata['SalePrice'])
        ax[plotindex].set(xlabel=i, ylabel='SalePrice')
        plotindex = plotindex+1

In [ ]:
fig,ax = plt.subplots()
fig.set_size_inches(30, 30) 
sb.heatmap(Cdata.corr(),cmap ="YlGnBu",linewidths = 0.1, annot = True)
top, bottom = ax.get_ylim()
ax.set_ylim(top+0.5, bottom-0.5)
plt.show()

In [ ]:
plotindex = 0
fig,ax = plt.subplots(1,2,figsize=(15,10))
for index,i in enumerate(cat): 
    if  plotindex <2:
        sb.boxplot(Cdata[i],Cdata['SalePrice'],ax=ax[plotindex])
        ax[plotindex].set(xlabel=i, ylabel='SalePrice')
        plotindex = plotindex+1
    else:
        plotindex=0
        fig,ax = plt.subplots(1,2,figsize=(15,10))
        sb.boxplot(Cdata[i],Cdata['SalePrice'],Cdata[i],ax=ax[plotindex])
        ax[plotindex].set(xlabel=i, ylabel='SalePrice')
        plotindex = plotindex+1
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean')
Cdata[con] = si.fit_transform(Cdata[con])

In [ ]:
Cdata = Cdata.join(pd.get_dummies(Cdata[cat],drop_first=True))


In [ ]:
Cdata = Cdata.drop(columns=cat)

In [ ]:
Cdata.info()

In [ ]:
#Outlier Analysis using inter Quantile range
outlier_data_quantile = []
for index,i in enumerate(con): 
    sorted(Cdata[i])
    quantile1, quantile3 = np.percentile(Cdata[i],[25,75])
    iqr=quantile3-quantile1
    lower_bound_val = quantile1 -(1.5 * iqr) 
    upper_bound_val = quantile3 +(1.5 * iqr)
    for val in Cdata[i]:
        if val < lower_bound_val or val > upper_bound_val:
            if i not in outlier_data_quantile:
                outlier_data_quantile.append(i)    
                
print(outlier_data_quantile)

In [ ]:
#Outlier Analysis using inter Quantile range
outlier_data_quantile = []
for index,i in enumerate(con): 
    print(i)
    sorted(Cdata[i])
    quantile1, quantile3 = np.percentile(Cdata[i],[25,75])
    iqr=quantile3-quantile1
    lower_bound_val = quantile1 -(1.5 * iqr) 
    upper_bound_val = quantile3 +(1.5 * iqr)
    print("columns name : {} ,quantile1 : {}, quantile3: {} ,iqr: {} ,lower_bound: {} , uppper bound: {} ".format(i,quantile1,quantile3,iqr,lower_bound_val,upper_bound_val))
    Cdata[i][Cdata[i] <= lower_bound_val] = lower_bound_val
    Cdata[i][Cdata[i] >= upper_bound_val] = upper_bound_val 

In [ ]:

Cdata.shape
Cdata.describe()

In [ ]:
con.remove('SalePrice')

In [ ]:
from sklearn.model_selection import train_test_split
X = Cdata.drop(columns=['SalePrice'])
Y = Cdata[['SalePrice']]
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.2,random_state=17)


In [ ]:
#Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain[con] = scaler.fit_transform(xtrain[con])
xtest[con] = scaler.transform(xtest[con])

In [ ]:
#UsingANN

train_x = np.asarray(xtrain)
test_x = np.asarray(xtest)
train_y = np.asarray(ytrain)
test_t = np.asarray(ytest)

import tensorflow as tf

In [ ]:
#iniitialization 

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))
ann.add(tf.keras.layers.Dense(units=1))

In [ ]:
#Compiling the ANN
ann.compile(optimizer='adam',loss= 'mean_squared_error',metrics=['mean_absolute_error'])
ann.fit(train_x,train_y,batch_size=32,epochs=100)

In [ ]:
pred = ann.predict(test_x)

In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score

print(mean_absolute_error(ytest,pred))
print(r2_score(ytest,pred))

In [ ]:
# Using Ridge & Lasso Regression

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

lasso_params = {'alpha':range(1,100)}
ridge_params = {'alpha':range(1,150)}

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
model4 = lr.fit(xtrain,ytrain)
pred_lr = model4.predict(xtest)

from sklearn.metrics import mean_absolute_error,r2_score

print("Linear Regression Mean Absolute Error",mean_absolute_error(ytest,pred_lr))
print("R2_Score",r2_score(ytest,pred_lr))

from sklearn.linear_model import Ridge,Lasso

Rlr = Ridge()
Llr = Lasso()

cvs = GridSearchCV(Llr,param_grid=lasso_params,cv=4,scoring= 'neg_mean_absolute_error')
cvmodel=cvs.fit(xtrain,ytrain)
print(cvmodel.best_estimator_)


cvs1 = GridSearchCV(Rlr,param_grid=ridge_params,cv=4,scoring= 'neg_mean_absolute_error')
cvmodel1=cvs1.fit(xtrain,ytrain)
print(cvmodel1.best_estimator_)

Rlr1 = Ridge(alpha=12)
model5 = Rlr1.fit(xtrain,ytrain)
pred_ridge = model5.predict(xtest)

print("Ridge  Regression Mean Absolute Error",mean_absolute_error(ytest,pred_ridge))
print("R2_Score",r2_score(ytest,pred_ridge))

Llr1 = Lasso(alpha=93)
model6 = Llr1.fit(xtrain,ytrain)
pred_lasso = model6.predict(xtest)

print("Lasso Regression Mean Absolute Error",mean_absolute_error(ytest,pred_lasso))
print("R2_Score",r2_score(ytest,pred_lasso))



In [ ]:
print("Ridge : {}%".format(model5.score(xtest,ytest)*100))
print("Lasso : {}%".format(model6.score(xtest,ytest)*100))